In [ ]:
from dask.distributed import Client, wait
from dask_cuda import DGX
from dask_cuda.initialize import initialize

In [ ]:
enable_tcp_over_ucx = True
enable_infiniband = False
enable_nvlink = True

In [ ]:
initialize(
    create_cuda_context=True,
    enable_tcp_over_ucx=enable_tcp_over_ucx,
    enable_infiniband=enable_infiniband,
    enable_nvlink=enable_nvlink,
)

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import DGX

# control number of workers with `CUDA_VISIBLE_DEVICES`
# definition
cluster = DGX(CUDA_VISIBLE_DEVICES=[0,1,2,3,4,5,6,7],
              enable_tcp_over_ucx=enable_tcp_over_ucx,
              enable_infiniband=enable_infiniband,
              enable_nvlink=enable_nvlink,
              dashboard_address=':8787')
client = Client(cluster);
client

In [ ]:
import dask.array as da
import dask.dataframe as dd

n_rows = 1000000000
n_keys = 5000000

left = dd.concat([
    da.random.random(n_rows).to_dask_dataframe(columns='x'),
    da.random.randint(0, n_keys, size=n_rows).to_dask_dataframe(columns='id'),
], axis=1).persist()
left

In [ ]:
n_rows = 10000000

right = dd.concat([
    da.random.random(n_rows).to_dask_dataframe(columns='y'),
    da.random.randint(0, n_keys, size=n_rows).to_dask_dataframe(columns='id'),
], axis=1).persist()
right

In [ ]:
import cudf, dask

In [ ]:
gleft = left.map_partitions(cudf.from_pandas)
gright = right.map_partitions(cudf.from_pandas)
# gright = gright.repartition(npartitions=8)
# gleft = gleft.repartition(npartitions=8)
gleft, gright = dask.persist(gleft, gright)  # persist data in device memory

In [ ]:
from distributed.utils import format_bytes
right_gb = format_bytes(len(gright) * 8 * 2)
left_gb = format_bytes(len(gleft) * 8 * 2)
print(f'Total Left: {left_gb} Total Right: {right_gb}')

In [ ]:
out = gleft.merge(gright, on=['id'])  # this is lazy

In [ ]:
out = out.persist()
%time _ = wait(out)

In [ ]:
cluster.scheduler.bandwidth

In [ ]:
data

In [ ]:
from distributed.utils import format_bytes
total = sum([d for w, d in data.items()])
format_bytes(total)